In [1]:
import threading

import redis
from datetime import timedelta
from itertools import zip_longest

# con = redis.StrictRedis(host='kepler-redis.6jjkdo.ng.0001.euw1.cache.amazonaws.com', port=6379, db=9)
# count = 0
# keys_changed = 0

from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
import multiprocessing as mp


def batcher(iterable, n):
    try:
        args = [iter(iterable)] * n
        return zip_longest(*args)
    except Exception as e:
        print(e)
        return []


def split_into_slices(sequence, slice_size):
    for i in range(0, len(sequence), slice_size):
        yield sequence[i, i + slice_size]


# for key_batch in batcher(con.scan_iter('*', count=50000), 50000):
#     # print(len(key_batch))
#
#     with con.pipeline() as pipeline:
#         pipeline.multi()
#         for key in key_batch:
#             if not key:
#                 break
#             if 'olxtr-30' not in key.decode('utf-8'):
#                 pipeline.expire(key, timedelta(days=30))
#                 keys_changed += 1
#         pipeline.execute()
#         print("Batch successfully updated batch: ", count)
#     count += 1
# print("Total keys updated: ", keys_changed)


def task_to_be_done(data):
    print(data)
    return "done"


# for p in batcher([1, 2, 3, 4, 5], 2):
#     print(p)


cpu_count = mp.cpu_count()

with ThreadPoolExecutor(max_workers=cpu_count * 4) as executor:
    tasks = [
        executor.submit(task_to_be_done, p) for p in batcher([1, 2, 3, 4, 5], 2)
    ]

    # print(tasks[0].done())  # False
    # print(tasks[1].done())  # False

    for task in tasks:
        print(task.result())

    # for task in tasks:
    #     task.add_done_callback(call_back_function)

pool = mp.Pool(cpu_count)

result = []

for p in batcher([1, 2, 3, 4, 5], 2):
    result.append(
        pool.apply(task_to_be_done, (p,))
    )

print(result)

queue = mp.Queue()


def do_task(queue):
    while True:
        item = queue.get()
        if item:
            # proces item
            # print(mp.current_process().name, item)
            print(threading.current_thread().name, item)


# processes = []
# for _ in range(cpu_count):
#     process = mp.Process(target=do_task, args=(queue,))
#     process.start()
#     processes.append(process)


# for p in batcher([1, 2, 3, 4, 5], 2):
#     queue.put(p)


from queue import Queue

q = Queue()


threads = []
for _ in range(cpu_count*4):

    thread = threading.Thread(target=do_task, args=(q, ))
    thread.start()


for p in split_into_slices([1, 2, 3, 4, 5], 2):
    q.put(p)


for thread in threads:
    thread.join()


In [ ]:
import threading
import multiprocessing as mp
from queue import Queue

cpu_count = mp.cpu_count()
q = Queue()

def read_file_in_chunk(fp, size):

    while True:
        data = fp.read(size=size)
        if not data:
            break

        yield data

def task_to_do():
    while True:
        data = q.get()
        if data:
            print(data)
            

threads = []
for _ in range(cpu_count*4):
    thread = threading.Thread(target=task_to_do)
    thread.start()
    threads.append(thread)
    

with open('file_name.txt', mode='rb') as f:
    for file_data in read_file_in_chunk(f, 1024):
        q.put(file_data)

for thread in threads:
    thread.join()


In [ ]:
tasks = []
with open('file_name.txt', mode='rb') as f:
    with ThreadPoolExecutor(max_workers=cpu_count*4) as executor:
        for file_data in read_file_in_chunk(f, 1024):
            tasks.append(executor.submit(task_to_do, arg = (file_data, )))

for task in tasks:
    print(tasks.result())

def call_back_function(future):
    print(future.result())

for task in tasks:
    task.add_done_callback(call_back_function)

In [4]:
slot = [1,1,2,3]
min_batt = 1

In [5]:
for i in slot:
    i -= min_batt
slot

[1, 1, 2, 3]

In [7]:

from sortedcontainers import SortedList

s = SortedList()


In [8]:
s

SortedList([])

In [9]:
s.bisect_left(5)

0

In [11]:
from sortedcontainers import SortedList

class Solution:
    def minAbsoluteDifference(self, nums, x: int) -> int:
        if x == 0: return 0
        res = float('inf')
        S = SortedList()
        for i in range(len(nums)):
            j = S.bisect_left(nums[i])
            # two options from here on
            # nums[i] - S[j-1]
            # S[j] - nums[i]
            print("j:", j)
            if j < len(S):
                res = min(res, abs(S[j]-nums[i]))
            if j > 0:
                res = min(res, abs(S[j-1]-nums[i]))
            if i >= x-1:
                S.add(nums[i-(x-1)])
            print("S", S)
        return res

In [13]:
a = Solution().minAbsoluteDifference([4,3,2,4], 2)
print(a)

j: 0
S SortedList([])
j: 0
S SortedList([4])
j: 0
S SortedList([3, 4])
j: 1
S SortedList([2, 3, 4])
0


In [20]:
def minimumDifference(nums) -> int:
    mem = {}
    n = len(nums)/2
    def dfs(i, diff, n1, n2):
        if i == len(nums):
            if n1 == n2 == n:
                return abs(diff)
            return float('inf')

        if (i, n1, n2) in mem:
            print("memused")
            return mem[(i, n1, n2)]

        min_diff = float('inf')

        if n1 == n:
            min_diff = dfs(i+1, diff-nums[i], n1, n2+1)
        elif n2 == n:
            min_diff = dfs(i+1, diff+nums[i], n1+1, n2)
        else:
            min_diff = min(dfs(i+1, diff+nums[i], n1+1, n2), dfs(i+1, diff-nums[i], n1, n2+1))

        mem[i] = min_diff
        return min_diff
    return dfs(0, 0, 0, 0)
a = minimumDifference([7772197,4460211,-7641449,-8856364,546755,-3673029,527497,-9392076,3130315,-5309187,-4781283,5919119,3093450,1132720,6380128,-3954678,-1651499,-7944388,-3056827,1610628,7711173,6595873,302974,7656726,-2572679,0,
                                  2121026,-5743797,-8897395,-9699694])
print(a)

KeyboardInterrupt: 